In [1]:
import import_ipynb
from Functions import *
from skimage.metrics import structural_similarity as ssim
from scipy.stats import entropy
import warnings
warnings.filterwarnings("ignore")
from numba import cuda
import numba

importing Jupyter notebook from Functions.ipynb


## Display Frames in List

In [2]:
def ThroughFrames(frames):
    i = 0
    while True:
        cv2.imshow('Frame', frames[i])
         # Wait for a key press to move to the next frame
        key = cv2.waitKeyEx(0)
        if key == ord('q'):
            break
        if key == 2424832:  # Left arrow key
            i = i - 1
            if i<0:
                i = 0
        if key == 2555904:  # Right arrow key
            i = i + 1
            if i>(len(frames)-1):
                i = len(frames)-1
    # Release the video capture object
    cv2.destroyAllWindows()

##  Entropy

In [3]:
@numba.jit
def Entropy(image):
    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Calculate the histogram of the grayscale image
    hist, _ = np.histogram(gray_image, bins=256, range=(0, 256), density=True)
    # Calculate the entropy
    hist_entropy = entropy(hist, base=2)
    return hist_entropy

## Temporal Signal to Noise Ratio for Inconsistency

In [4]:
@numba.jit
def TSNR(image1, image2, image3):  
    if image1 is None or image2 is None:
        raise ValueError("One or both image paths are invalid")
    frame_diff = cv2.absdiff(image1, image2)
    mean_diff = np.mean(frame_diff)
    std_diff = np.std(frame_diff)
    tsnr = mean_diff / (std_diff + 1e-10)
    return abs(1-tsnr)

## Absolute Difference

In [5]:
@numba.jit
def Abs_Dif(image1, image2, image3):
    if image1 is None or image2 is None:
        raise ValueError("One or both image paths are invalid")
    frame_diff = cv2.absdiff(image1, image2)
    tci = np.mean(frame_diff)
    return tci

## Optical Flow End Point Error

In [6]:
@numba.jit
def OF_EPE(image1, image2, image3):
    if image1 is None or image2 is None:
        raise ValueError("One or both image paths are invalid")
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    # EPE (End Point Error)
    epe = np.linalg.norm(flow, axis=2).mean()
    return epe

## Oprical Flow Angular Error

In [7]:
@numba.jit
def OF_AE(image1, image2, image3):
    if image1 is None or image2 is None:
        raise ValueError("One or both image paths are invalid")
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    # AE (Angular Error)
    u, v = flow[:,:,0], flow[:,:,1]
    magnitude, angle = cv2.cartToPolar(u, v)
    ae = np.mean(angle)
    return ae

## Optical Flow Error

In [ ]:
@numba.jit
def OF(img1, img2, img3):
    # Check if images are the same size
    if img1.shape != img2.shape or img2.shape != img3.shape:
        raise ValueError("All images must be of the same size")
    # Apply Canny edge detection
    edges1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    edges2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    edges3 = cv2.cvtColor(img3, cv2.COLOR_BGR2GRAY)
    # Compute optical flow between the first and second images
    flow1 = cv2.calcOpticalFlowFarneback(edges1, edges2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    # Compute optical flow between the second and third images
    flow2 = cv2.calcOpticalFlowFarneback(edges2, edges3, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    # Compute the magnitude and angle of the optical flow vectors for both flows
    magnitude1 = np.sqrt(flow1[..., 0]**2 + flow1[..., 1]**2)
    angle1 = np.arctan2(flow1[..., 1], flow1[..., 0]) 
    magnitude2 = np.sqrt(flow2[..., 0]**2 + flow2[..., 1]**2)
    angle2 = np.arctan2(flow2[..., 1], flow2[..., 0])
    # Compute the difference between the two optical flows
    magnitude_diff = np.abs(magnitude1 - magnitude2)
    angle_diff = np.abs(angle1 - angle2)
    # Normalize differences
    magnitude_index = np.sum(magnitude_diff) / (magnitude1.size + 1e-6)  # Avoid division by zero
    angle_index = np.sum(angle_diff) / (angle1.size + 1e-6)  # Avoid division by zero
    # Combine magnitude and angle differences
    difference_index = magnitude_index + angle_index
    return difference_index

## Optical Flow Border Inconsistency Index

In [ ]:
@numba.jit
def OF_B(img1, img2, img3):
    # Check if images are the same size
    if img1.shape != img2.shape or img2.shape != img3.shape:
        raise ValueError("All images must be of the same size")
    # Apply Canny edge detection
    edges1 = cv2.Canny(img1, 100, 200)
    edges2 = cv2.Canny(img2, 100, 200)
    edges3 = cv2.Canny(img3, 100, 200)
    # Compute optical flow between the first and second images
    flow1 = cv2.calcOpticalFlowFarneback(edges1, edges2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    # Compute optical flow between the second and third images
    flow2 = cv2.calcOpticalFlowFarneback(edges2, edges3, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    # Compute the magnitude and angle of the optical flow vectors for both flows
    magnitude1 = np.sqrt(flow1[..., 0]**2 + flow1[..., 1]**2)
    angle1 = np.arctan2(flow1[..., 1], flow1[..., 0]) 
    magnitude2 = np.sqrt(flow2[..., 0]**2 + flow2[..., 1]**2)
    angle2 = np.arctan2(flow2[..., 1], flow2[..., 0])
    # Compute the difference between the two optical flows
    magnitude_diff = np.abs(magnitude1 - magnitude2)
    angle_diff = np.abs(angle1 - angle2)
    # Normalize differences
    magnitude_index = np.sum(magnitude_diff) / (magnitude1.size + 1e-6)  # Avoid division by zero
    angle_index = np.sum(angle_diff) / (angle1.size + 1e-6)  # Avoid division by zero
    # Combine magnitude and angle differences
    difference_index = magnitude_index + angle_index
    return difference_index

## Gray Scale Absolute Difference

In [8]:
@numba.jit
def Gray_Dif(image1, image2, image3):
    if image1 is None or image2 is None:
        raise ValueError("One or both image paths are invalid")
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    diff = cv2.absdiff(gray1, gray2)
    return np.mean(diff)

## Temporal Structural Similarity Index Measure for Inconsistency

In [9]:
@numba.jit
def TSSIM(image1, image2, image3):
    if image1 is None or image2 is None:
        raise ValueError("One or both image paths are invalid")
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    ssim_value = ssim(gray1, gray2,multichannel=True,win_size=3)
    return abs(1-ssim_value)

## Mean Squared Error

In [10]:
@numba.jit
def MSE(image1, image2, image3):
    if image1 is None or image2 is None:
        raise ValueError("One or both image paths are invalid")
    gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    mse = np.mean((gray1 - gray2) ** 2)
    return mse

## Border Error

In [11]:
@numba.jit
def Border_Err(image1, image2, image3):
    lower_threshold = 50
    upper_threshold = 255
    if len(image1.shape) > 1:
        image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    if len(image2.shape) > 1:
        image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    # Apply Canny edge detection
    canny1 = cv2.Canny(image1, lower_threshold, upper_threshold)
    canny2 = cv2.Canny(image2, lower_threshold, upper_threshold)
    # Calculate the absolute difference between the two Canny images
    abs_diff = cv2.absdiff(canny1, canny2)
    return np.mean(abs_diff)

## Color Range Consistency

In [12]:
@numba.jit
def CRC(image1, image2, image3):
    if image1 is None or image2 is None:
        raise ValueError("One or both image paths are invalid")
    image1 = image1.astype(np.float64)
    image2 = image2.astype(np.float64)
    diff_r_min = abs(np.min(image1[:,:,2]) - np.min(image2[:,:,2]))  # Red channel min
    diff_r_max = abs(np.max(image1[:,:,2]) - np.max(image2[:,:,2]))  # Red channel max
    diff_g_min = abs(np.min(image1[:,:,1]) - np.min(image2[:,:,1]))  # Green channel min
    diff_g_max = abs(np.max(image1[:,:,1]) - np.max(image2[:,:,1]))  # Green channel max
    diff_b_min = abs(np.min(image1[:,:,0]) - np.min(image2[:,:,0]))  # Blue channel min
    diff_b_max = abs(np.max(image1[:,:,0]) - np.max(image2[:,:,0]))  # Blue channel max
    crci = (np.mean(diff_r_min) + np.mean(diff_r_max) + np.mean(diff_g_min) + np.mean(diff_g_max) + np.mean(diff_b_min) + np.mean(diff_b_max)) / 6
    return crci

## Entropy Difference

In [13]:
@numba.jit
def Entropy_Dif(image1,image2, image3):
    return abs(Entropy(image1)-Entropy(image2))

# Frequency Magnitude Difference

In [ ]:
@numba.jit
def Freq_Dif(img1, img2, img3):
    # Check if images are the same size
    if img1.shape != img2.shape:
        raise ValueError("Images must be of the same size")
    # Compute FFT
    f1 = np.fft.fft2(img1)
    f2 = np.fft.fft2(img2)
    # Shift zero frequency component to the center
    f1_shifted = np.fft.fftshift(f1)
    f2_shifted = np.fft.fftshift(f2)
    # Compute magnitude spectrum
    magnitude1 = np.abs(f1_shifted)
    magnitude2 = np.abs(f2_shifted)
    # Compute the difference in magnitude spectra
    magnitude_diff = np.abs(magnitude1 - magnitude2)
    # Compute the difference index
    difference_index = np.sum(magnitude_diff) / np.sum(magnitude1 + magnitude2)
    return difference_index

## Combined All Metrics

In [14]:
def Combined_Metrics(image1, image2,op):
    if image1 is None or image2 is None:
        raise ValueError("One or both image paths are invalid")
    tsnr = TSNR(image1,image2)
    adif = Abs_Dif(image1,image2)
    epe = OF_EPE(image1,image2)
    ae = OF_AE(image1,image2)
    gray = Gray_Dif(image1,image2)
    ssim_value = TSSIM(image1,image2)
    mse_value = MSE(image1,image2)
    border_consistency_value = Border_Err(image1,image2)
    crci_value = CRC(image1,image2)
    ent = Entropy_Dif(image1,image2)
    # Normalize and combine the metrics into a single consistency index
    metrics = np.array([tsnr,adif, epe, ae, gray,ssim_value, mse_value, border_consistency_value, crci_value, ent])
    normalized_metrics = (metrics - np.min(metrics)) / (np.max(metrics) - np.min(metrics))
    combined_consistency_index = np.mean(normalized_metrics)
    #print(metrics)
    # Z-score normalization
    mean = np.mean(metrics)
    std = np.std(metrics)
    Z_metrics = (metrics - mean) / std
    
    if op=="norm":
        return combined_consistency_index
    if op=="mean":
        return np.mean(metrics)
    if op=="log":
        metrics[metrics <= 0] = 1e-10
        return np.sum(np.log(metrics))
    if op=="Z":
        return np.mean(Z_metrics)

## Mixed Metrics

In [15]:
@numba.jit
def Mix_Metrics(image1, image2,image3,op="mean",M = [TSSIM,MSE],W=[1,1]):
    if image1 is None or image2 is None:
        raise ValueError("One or both image paths are invalid")
    metrics = []
    for i in range(len(M)):
        #print(image1.shape,image2.shape)
        metrics.append(W[i]*M[i](image1,image2,image3))
    # Normalize and combine the metrics into a single consistency index
    metrics = np.array(metrics)
    normalized_metrics = (metrics - np.min(metrics)) / (np.max(metrics) - np.min(metrics))
    combined_consistency_index = np.mean(normalized_metrics)
    #print(metrics)
    # Z-score normalization
    mean = np.mean(metrics)
    std = np.std(metrics)
    Z_metrics = (metrics - mean) / std
    
    if op=="norm":
        return combined_consistency_index
    if op=="mean":
        return np.mean(metrics)
    if op=="log":
        metrics[metrics <= 0] = 1e-10
        return np.sum(np.log(metrics))
    if op=="Z":
        return np.mean(Z_metrics)

## WIndowed Max Inconsistency

In [16]:
@numba.jit
def WMax_Inconsistency(img1, img2, img3, wsize=(3,3), step=(3,3), Func=Combined_Metrics, op="mean",M=[],Weights=[]):
    kw,kh = wsize
    H,W,_ = img1.shape
    sw,sh = step
    maxC = 0
    maxR = [0,0,kw,kh]
    # Calculate the output dimensions
    output_height = (H - kh) // sh + 1
    output_width = (W - kw) // sw + 1
    # Initialize the output array
    result = np.zeros((output_height, output_width))
    # Perform the convolution
    for i in range(0, output_height*sh, sh):
        #print('Row: ',i,'/',output_height*sh,end='\r')
        for j in range(0, output_width*sw, sw):
            # Extract the region of interest
            region1 = img1[i:i + kh, j:j + kw]
            region2 = img2[i:i + kh, j:j + kw]
            region3 = img3[i:i + kh, j:j + kw]
            # Perform element-wise multiplication and sum the result
            if Func==Combined_Metrics or Func==Mix_Metrics:
                if len(M)>0 and Func==Mix_Metrics:
                    result[i // sh, j // sw] = Func(region1, region2, region3,op,M,Weights)
                else:
                    result[i // sh, j // sw] = Func(region1, region2,region3,op)
            else:
                result[i // sh, j // sw] = Func(region1, region2,region3)
            if result[i//sh,j//sw]>=maxC:
                maxR = [i,i+kh,j,j+kw] 
                maxC = result[i//sh,j//sw]
            if (j+sw+kw)>W:
                break
        if (i+sh+kh)>H:
            break
    #print('')
    return result,maxR

## Video Consistency

In [17]:
def Vid_consistency(F,Func=Combined_Metrics,op="mean",M=[],W=[]):
    C = []
    for i in range(len(F)-1):
        if Func==Combined_Metrics or Func==Mix_Metrics:
            if len(M)>0 and Func==Mix_Metrics:
                C.append(Func(F[i],F[i+1],op,M,W))
            else:
                C.append(Func(F[i],F[i+1],op))
        else:
            C.append(Func(F[i],F[i+1]))
    return np.mean(np.asarray(C))

## Windowd Video Consistency

In [18]:
def Vid_consistency_W(F,wsize=(3,3),step=(3,3),Func=Combined_Metrics,op="mean",M=[],W=[]):
    C = []
    for i in range(len(F)-1):
        print('Frame: ',i+1,'/',len(F)-1,end='\r')
        Metrics,Region = WMax_Inconsistency(F[i],F[i+1],wsize,step,Func,op,W)
        C.append(np.mean(Metrics))
    return np.mean(np.asarray(C))

## Draw Window with Max Inconsistency

In [19]:
@numba.jit
def DrawInconsistancy(F,wsize=(50,50),step=(50,50),Func=Combined_Metrics,op="mean",M=[],W=[]):
    L = 3
    DI = np.copy(F)
    for i in range(len(F)-2):
        print('Frame: ',i+1,'/',len(F)-1,end='\r')
        Metrics,Region = WMax_Inconsistency(F[i],F[i+1],F[i+2],wsize,step,Func,op,M,W)
        #print((Region[0],Region[2]),(Region[1],Region[3]))
        DI[i+1][Region[0]:Region[0]+L,Region[2]:Region[3]] = [0,255,0] 
        DI[i+1][Region[0]:Region[1],Region[2]:Region[2]+L] = [0,255,0] 
        DI[i+1][Region[1]:Region[1]+L,Region[2]:Region[3]] = [0,255,0] 
        DI[i+1][Region[0]:Region[1],Region[3]:Region[3]+L] = [0,255,0] 
        DI[i+1] = cv2.putText(DI[i+1],str(np.mean(Metrics)),(10,20),cv2.FONT_HERSHEY_SIMPLEX,0.25,(0,0,255),1,cv2.LINE_AA) 
    return DI

## Draw Window with Max Inconsistency of Diffrent Window Sizes

In [20]:
#@numba.jit
def DrawInconsistancy1(F,Func=Combined_Metrics,op="mean",M=[],W=[]):
    L = 3
    DI = np.copy(F)
    for i in range(len(F)-2):
        print('Frame: ',i+1,'/',len(F)-1,end='\r')
        R = -1000
        Region = [0,0,0,0]
        for j in range(10,100,10):
            Metrics,Reg= WMax_Inconsistency(F[i],F[i+1],F[i+2],(j,j),(j//2,j//2),Func,op,M,W)
            if np.mean(Metrics)>R:
                R = np.mean(Metrics)
                Region = Reg
        #print((Region[0],Region[2]),(Region[1],Region[3]))
        
        DI[i+1][Region[0]:Region[0]+L,Region[2]:Region[3]] = [0,255,0] 
        DI[i+1][Region[0]:Region[1],Region[2]:Region[2]+L] = [0,255,0] 
        DI[i+1][Region[1]:Region[1]+L,Region[2]:Region[3]] = [0,255,0] 
        DI[i+1][Region[0]:Region[1],Region[3]:Region[3]+L] = [0,255,0] 
        DI[i+1] = cv2.putText(DI[i+1],str(R),(10,20),cv2.FONT_HERSHEY_SIMPLEX,0.25,(0,0,255),1,cv2.LINE_AA) 
    return DI

In [21]:
@numba.jit
def InconsistentRegion(F,Func=Combined_Metrics,op="mean",M=[],W=[]):
    L = 3
    h,w,_ = F[0].shape
    DI = [np.zeros((h,w)) for _ in F]
    for i in range(len(F)-2):
        #print('Frame: ',i+1,'/',len(F)-1,end='\r')
        R = -1000
        Region = [0,0,0,0]
        for j in range(20,100,20):
            Metrics,Reg= WMax_Inconsistency(F[i],F[i+1],F[i+2],(j,j),(j//2,j//2),Func,op,M,W)
            if np.mean(Metrics)>R:
                R = np.mean(Metrics)
                Region = Reg
        #print((Region[0],Region[2]),(Region[1],Region[3]))
        
        DI[i+1][Region[0]:Region[1],Region[2]:Region[3]] = 255
    return DI,R

# Experiments

F = read_images("saved_frames")
F[3] = F[3][:,:,:3]
F[6] = F[6][:,:,:3]
F[9] = F[9][:,:,:3]
F[14] = F[14][:,:,:3]
I = DrawInconsistancy(F[:13],(50,50),(10,10),Mix_Metrics,"mean",[CRC,Border_Err,Abs_Dif],[2,1.5,10])

ThroughFrames(I)

F = read_images("saved_frames")
F[3] = F[3][:,:,:3]
F[6] = F[6][:,:,:3]
F[9] = F[9][:,:,:3]
F[14] = F[14][:,:,:3]
I = DrawInconsistancy1(F[:13],Mix_Metrics,"mean",[CRC,Border_Err],[1.5,1])

ThroughFrames(I)

In [22]:
#Check which values with min mean consistency and which max mean consistency. Ex: 1-ssim
#Weights of each metric
#Which metrics are the best?
#Look for change in entropy too

In [23]:
#Buscar origen del ruido
#Checar si el ruido proviene desde el origen
#Trabajar con fracuencias
#DNN para cada tipo de error
#Cambiar parametros de caricaturizacion por medio de DNN

#Lista de parametros e indices para identificar cuales son los mejores valores
#Algoritmos Geneticos
#For x,y in zip(X,Y) 

#Para indices de inconsistencia y para caricaturizacion
#CHecar con frames pasados (mean,var,etc) de cada parametro